In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import talib as tb
import yfinance as yf


import datetime
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import *
from datetime import timedelta

In [7]:


def df_converter(df): 
    df_sp500 = yf.download('^GSPC', 
                      start='2017-11-09', 
                      end='2022-05-08', 
                      progress=False)
    df_dollar = yf.download('DX=F',  
                      progress=False)
    # clearing dollar and sp500 df
    df_dollar.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_dollar.rename(columns={"Close": "dollar_close"}, inplace=True)
    df_sp500.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)
    df_sp500.rename(columns={"Close": "sp500_close"}, inplace=True)
    # clearing general df
    #df_eth.drop('Unnamed: 0', axis=1, inplace=True)
    #df.drop('adj_close', axis=1, inplace=True)
    df.index = df.index.astype('datetime64[ns]')
    # MA df
    df_ma = df['Close'].to_frame()
    df_ma['SMA30'] = df_ma['Close'].rolling(15).mean()
    df_ma['CMA30'] = df_ma['Close'].expanding().mean()
    df_ma['EMA30'] = tb.EMA(df_ma['Close'], timeperiod=15)
    df_ma.dropna(inplace=True)
    # Stoch df
    slowk, slowd = tb.STOCH(df["High"], df["Low"], df["Close"], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df_stoch = pd.DataFrame(index=df.index,
                                data={"slowk": slowk,
                                    "slowd": slowd})
    df_stoch.dropna(inplace=True)
    # for later use in the concat
    stoch_c = ['slowk', 'slowd']
    # MACD df 
    macd, macdsignal, macdhist = tb.MACD(df.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    df_macd = pd.DataFrame(index=df.index,
                            data={"macd": macd,
                                  "macdsignal": macdsignal,
                                  "macdhist": macdhist})
    df_macd.dropna(inplace=True)
    # for later use in the concat
    macd_c = ['macd', 'macdsignal', 'macdhist']
    # bb df
    upper, middle, lower = tb.BBANDS(df["Close"], timeperiod=15)
    df_bands = pd.DataFrame(index=df.index,
                                data={"bb_low": lower,
                                    "bb_ma": middle,
                                    "bb_high": upper})
    df_bands.dropna(inplace=True)
    # for later use in the concat
    bands_c = ['bb_low', 'bb_ma', 'bb_high']
    # rsi df
    rsi = tb.RSI(df['Close'], timeperiod=15)
    df_rsi = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "rsi": rsi})

    df_rsi.dropna(inplace=True)
    #stdev df
    stdev = tb.STDDEV(df['Close'], timeperiod=15, nbdev=1)
    df_stdev = pd.DataFrame(index=df.index,
                            data={"close": df['Close'],
                                  "stdev": stdev})
    df_stdev.dropna(inplace=True)
    # adx df
    adx = tb.ADX(df['High'], df['Low'], df['Close'], timeperiod=15)
    df_adx = pd.DataFrame(index=df.index,
                                data={"close": df['Close'],
                                    "adx": adx})

    df_adx.dropna(inplace=True)

    # concat 
    result =pd.concat([df, df_ma[['SMA30','CMA30','EMA30']], df_adx['adx'], df_bands[bands_c], df_macd[macd_c], df_rsi['rsi'], df_stdev['stdev'], df_stoch[stoch_c], df_dollar['dollar_close'], df_sp500['sp500_close']], axis=1)
    result.fillna(method='ffill', inplace=True)
    result.dropna(inplace=True)

    return result 

In [8]:
df_coin = yf.download('BTC-USD',
                      end=date.today(), 
                      progress=False,
    )

In [10]:
df_coin = df_converter(df_coin)

In [11]:
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3.226250e+09,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5.208250e+09,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4.908680e+09,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8.957350e+09,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6.263250e+09,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-16,31304.375000,31305.341797,29251.884766,29862.917969,29862.917969,3.261390e+10,33200.786719,12201.112625,32649.327556,40.123429,...,40596.192649,-2825.319270,-2501.113459,-324.205811,32.717422,3697.702965,72.797664,58.885824,104.199997,4123.339844
2022-05-17,29862.408203,30694.490234,29570.302734,30425.857422,30425.857422,2.910147e+10,32660.555339,12207.621463,32371.393789,40.599029,...,39589.225018,-2736.028041,-2548.096376,-187.931666,35.085322,3464.334839,74.824425,69.740782,103.406998,4123.339844
2022-05-18,30424.478516,30618.716797,28720.271484,28720.271484,28720.271484,3.128527e+10,32058.543229,12213.516732,31915.003501,41.287032,...,38675.828410,-2770.948857,-2592.666872,-178.281985,31.487989,3308.642591,45.882390,64.501493,103.862000,4123.339844


In [26]:
df_coin.index

DatetimeIndex(['2017-11-09', '2017-11-10', '2017-11-11', '2017-11-12',
               '2017-11-13', '2017-11-14', '2017-11-15', '2017-11-16',
               '2017-11-17', '2017-11-18',
               ...
               '2022-05-10', '2022-05-11', '2022-05-12', '2022-05-13',
               '2022-05-14', '2022-05-15', '2022-05-16', '2022-05-17',
               '2022-05-18', '2022-05-19'],
              dtype='datetime64[ns]', name='Date', length=1653, freq='D')

In [27]:
start_date = '2021-10-22'

In [28]:
start_date = pd.to_datetime(start_date)

In [29]:
from datetime import datetime


In [30]:
start_date

Timestamp('2021-10-22 00:00:00')

In [31]:
df_coin

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2017-11-09,7446.830078,7446.830078,7101.520020,7143.580078,7143.580078,3226249984,6720.195964,1012.489040,6798.973215,37.428679,...,7929.899976,547.298468,521.809901,25.488567,65.836431,604.852006,33.184032,38.772059,94.351997,2584.620117
2017-11-10,7173.729980,7312.000000,6436.870117,6618.140137,6618.140137,5208249856,6767.749967,1017.359285,6776.369081,36.238180,...,7899.045668,482.149729,513.877867,-31.728137,55.203600,565.647850,30.022892,33.577955,94.278000,2582.300049
2017-11-11,6618.609863,6873.149902,6204.220215,6357.600098,6357.600098,4908680192,6806.196647,1021.994910,6724.022958,34.548539,...,7835.311479,404.828866,492.068066,-87.239201,50.841315,514.557416,13.668599,25.625174,94.278000,2582.300049
2017-11-12,6295.450195,6625.049805,5519.009766,5950.069824,5950.069824,8957349888,6819.328646,1026.269043,6627.278816,32.896648,...,7798.154128,307.126870,455.079827,-147.952957,44.895661,489.412741,14.127749,19.273080,94.278000,2582.300049
2017-11-13,5938.250000,6811.189941,5844.290039,6559.490234,6559.490234,6263249920,6846.371322,1031.063862,6618.805243,30.978407,...,7771.077715,275.694434,419.202748,-143.508315,53.591318,462.353197,27.607649,18.467999,94.388000,2584.840088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,30098.585938,31308.191406,29527.740234,31305.113281,31305.113281,25835372065,33774.531771,12194.800329,33047.386068,39.613857,...,41377.532033,-2840.874322,-2420.062007,-420.812316,35.843737,3801.500131,61.600258,43.103163,103.694000,4123.339844
2022-05-16,31304.375000,31305.341797,29251.884766,29862.917969,29862.917969,32613897286,33200.786719,12201.112625,32649.327556,40.123429,...,40596.192649,-2825.319270,-2501.113459,-324.205811,32.717422,3697.702965,72.797664,58.885824,103.694000,4123.339844
2022-05-17,29862.408203,30694.490234,29570.302734,30425.857422,30425.857422,29101473475,32660.555339,12207.621463,32371.393789,40.599029,...,39589.225018,-2736.028041,-2548.096376,-187.931666,35.085322,3464.334839,74.824425,69.740782,103.694000,4123.339844


In [32]:
df_coin = df_coin[df_coin.index >= start_date]

In [33]:
df_coin.head()

,Open,High,Low,Close,Adj Close,Volume,SMA30,CMA30,EMA30,adx,...,bb_high,macd,macdsignal,macdhist,rsi,stdev,slowk,slowd,dollar_close,sp500_close
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-22,62237.890625,63715.023438,60122.796875,60692.265625,60692.265625,38434082775,59411.947135,9548.177219,59279.606622,42.268428,...,66512.766057,4029.816080,3784.073284,245.742796,61.311059,3550.409461,45.656328,70.563090,93.625000,4544.899902
2021-10-23,60694.628906,61743.878906,59826.523438,61393.617188,61393.617188,26882546034,59906.998437,9568.163896,59543.857943,41.723052,...,66432.717203,3820.417900,3791.342207,29.075693,62.480777,3262.859383,23.700462,48.104014,93.625000,4544.899902
2021-10-24,61368.343750,61505.804688,59643.343750,60930.835938,60930.835938,27316183882,60304.505990,9587.956833,59717.230192,41.121085,...,66281.803561,3575.904948,3748.254755,-172.349808,61.173225,2988.648786,16.516932,28.624574,93.625000,4544.899902
2021-10-25,60893.925781,63729.324219,60691.800781,63039.824219,63039.824219,31064911614,60855.722396,9608.546920,60132.554446,41.093764,...,66179.665938,3511.822299,3700.968264,-189.145965,64.772807,2661.971771,29.515861,23.244418,93.808998,4566.479980
2021-10-26,63032.761719,63229.027344,59991.160156,60363.792969,60363.792969,34878965587,61047.655990,9628.090719,60161.459261,40.705583,...,66070.739534,3208.121750,3602.398961,-394.277211,57.522762,2511.541772,28.040026,24.690939,93.941002,4574.790039


# Predictions Graph

In [39]:
def add_days(df, forecast_length):
    end_point = len(df)
    df1 = pd.DataFrame(index=range(forecast_length), columns=range(2))
    df1.columns = ['Close', 'Date']
    df = df.append(df1)
    df = df.reset_index(drop=True)
    x = df.at[end_point - 1, 'Date']
    x = pd.to_datetime(x, format='%Y-%m-%d')
    for i in range(forecast_length):
        df.at[df.index[end_point + i], 'Date'] = x + timedelta(days=1+i)
        df.at[df.index[end_point + i], 'Close'] = 0
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df = df.drop(['Date'], axis=1)
    return df

In [40]:
def forecasting(model,df1, forecast_length):
    df3 = df1[['Close', 'Date']]
    df3 = add_days(df3, forecast_length)
    finaldf = df1.drop('Date', axis=1)
    finaldf = finaldf.reset_index(drop=True)
    end_point = len(finaldf)
    x = end_point - forecast_length
    finaldf_train = finaldf.loc[:x - 1, :]
    finaldf_train_x = finaldf_train.loc[:, finaldf_train.columns != 'Close']
    finaldf_train_y = finaldf_train['Close']

    fit = model.fit(finaldf_train_x, finaldf_train_y)
    yhat = []
    end_point = len(finaldf)
    df3_end = len(df3)
    for i in range(forecast_length, 0, -1):
        y = end_point - i
        inputfile = finaldf.loc[y:end_point, :]
        inputfile_x = inputfile.loc[:, inputfile.columns != 'Close']
        pred_set = inputfile_x.head(1)
        pred = fit.predict(pred_set)
        df3.at[df3.index[df3_end - i], 'Close'] = pred[0]
        finaldf = df1.drop('Date', axis=1)
        finaldf = finaldf.reset_index(drop=True)
        yhat.append(pred)
    yhat = np.array(yhat)
    return yhat

In [41]:
def predictions(df_coin,model, forecast_lenght = 5, train_lenght = 100,target = 'Close'):
    """ df_coin must be with date in index,
        forecast_lenght is the amount of days that we will predict
        model is the model predefined to use to get our predictions
        train_length is the amount of days that we will use to train the model
        target is what we are predicting
        This will return a graphic that will contain the data from the train set and our predictions
    """
    df_coin = df_coin.tail(train_lenght)
    df_coin.reset_index(inplace=True)
    df_coin['Date'] = pd.to_datetime(df_coin['Date'], format='%Y-%m-%d')

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=df_coin['Date'], y=df_coin[target], 
                    name='Actual Values', mode='lines',line=dict(color='black')))
    #Predictions
    forecast = forecasting(model,df_coin,forecast_lenght)
    #df that will contain the predictions
    df_pred = pd.DataFrame(columns=['Date',target])
    #Adding the predictions to our dataset
    for day, x in enumerate(forecast):
        new_row={'Date':df_coin['Date'].max() + timedelta(days=1+day),
     target:x[0]}
        df_pred = df_pred.append(new_row, ignore_index=True)

    df_pred['Date'] = pd.to_datetime(df_pred['Date'], format='%Y-%m-%d')

    fig2.add_trace(go.Scatter(x=df_pred['Date'], y=df_pred[target], name='Predictions', mode='lines',line=dict(color='red')))
    fig2.update_layout(dict(updatemenus=[
                        dict(
                        type = "buttons",
                        direction = "left",
                        buttons=list([
                                dict(
                                args=["visible", "legendonly"],
                                label="Deselect All",
                                method="restyle"
                                ),
                                dict(
                                args=["visible", True],
                                label="Select All",
                                method="restyle"
                                )
                        ]),
                        pad={"r": 10, "t": 10},
                        showactive=False,
                        x=1,
                        xanchor="right",
                        y=1.1,
                        yanchor="top"
                        ),
                ]
        ))
    fig2.show()

In [42]:
from sklearn.ensemble import RandomForestRegressor
rf_final = RandomForestRegressor(random_state=10,criterion='mae', max_depth=20, max_features='auto', n_estimators=30)

In [45]:
predictions(df_coin, rf_final, 5, 20, 'Close')

C:\Users\Admin\AppData\Local\Temp/ipykernel_10792/2344453812.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\Admin\anaconda3\envs\ML\lib\site-packages\sklearn\ensemble\_forest.py:395: FutureWarning:

Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.

